In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [3]:

query = "SELECT * FROM core_sale;"
df_sales = pd.read_sql(query, conn)

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:

df_sales_copy = df_sales.copy()
df_sales_copy

,id,income,date,description,client_id,salesman_id,income_currency,status
0,00034292,0.03,2018-11-14,,400,34,USD,PENDING
1,00034306,0.05,2018-11-16,,631,33,USD,PENDING
2,00034340,112.85,2018-11-26,,588,37,USD,PENDING
3,00034346,982.69,2018-11-26,,271,32,USD,PENDING
4,00034347,1611.32,2018-11-26,,250,31,USD,PENDING
...,...,...,...,...,...,...,...,...
21660,042087,45.44,2021-04-22,,178,42,USD,PENDING
21661,00003484,8.52,2019-11-21,,264,34,USD,PENDING
21662,NE00004937,45.44,2021-03-04,,178,34,USD,PENDING
21663,NE006477,38.00,2021-06-02,,1043,39,USD,PENDING


In [5]:
df_sales_copy["year"] = df_sales_copy.apply(lambda row : row["date"].year, axis=1)
df_sales_copy["month"] = df_sales_copy.apply(lambda row : row["date"].month, axis=1)
df_sales_copy["day"] = df_sales_copy.apply(lambda row : row["date"].day, axis=1)
df_sales_copy.drop(columns=["date", "description", "status", "income_currency"], inplace=True)
df_sales_copy

,id,income,client_id,salesman_id,year,month,day
0,00034292,0.03,400,34,2018,11,14
1,00034306,0.05,631,33,2018,11,16
2,00034340,112.85,588,37,2018,11,26
3,00034346,982.69,271,32,2018,11,26
4,00034347,1611.32,250,31,2018,11,26
...,...,...,...,...,...,...,...
21660,042087,45.44,178,42,2021,4,22
21661,00003484,8.52,264,34,2019,11,21
21662,NE00004937,45.44,178,34,2021,3,4
21663,NE006477,38.00,1043,39,2021,6,2


In [6]:
df_sales_by_client = df_sales_copy.copy()
df_sales_by_client.set_index(["year", "month", "day", "client_id"], inplace=True)
df_sales_by_client

id   income  salesman_id
year month day client_id                                  
2018 11    14  400          00034292     0.03           34
           16  631          00034306     0.05           33
           26  588          00034340   112.85           37
               271          00034346   982.69           32
               250          00034347  1611.32           31
...                              ...      ...          ...
2021 4     22  178            042087    45.44           42
2019 11    21  264          00003484     8.52           34
2021 3     4   178        NE00004937    45.44           34
     6     2   1043         NE006477    38.00           39
     4     21  831          NE005643    14.30           41

[21665 rows x 3 columns]

In [7]:
df_sales_client_by_salesman = df_sales_copy.copy()
df_sales_client_by_salesman.set_index(["salesman_id"], inplace=True)
df_sales_client_by_salesman

,id,income,client_id,year,month,day
salesman_id,,,,,,
34,00034292,0.03,400,2018,11,14
33,00034306,0.05,631,2018,11,16
37,00034340,112.85,588,2018,11,26
32,00034346,982.69,271,2018,11,26
31,00034347,1611.32,250,2018,11,26
...,...,...,...,...,...,...
42,042087,45.44,178,2021,4,22
34,00003484,8.52,264,2019,11,21
34,NE00004937,45.44,178,2021,3,4


In [8]:
salesman_count = df_sales_copy.loc[df_sales_copy["salesman_id"] == 36]["id"].count()
salesman_count

631

In [9]:
client_count = df_sales_copy.loc[df_sales_copy["client_id"] == 250]["id"].count()
client_count

45

In [10]:
client_per_salesman = df_sales_copy.loc[df_sales_copy["salesman_id"] == 36]
client_per_salesman_count = client_per_salesman.loc[client_per_salesman["client_id"] == 250]


dumb_df = client_per_salesman.copy() 
dumb_df.reset_index(inplace=True)
dumb_df.drop(columns=["year", "month", "day", "income", "index"], inplace=True)

pivot_table = dumb_df.pivot_table(index="salesman_id", columns="client_id", aggfunc="count")
pivot_table

# pivot_table.max(axis=1)
# pivot_table.reset_index(inplace=True)
# pivot_table
# max_index = pivot_table.idxmax(axis=1)
# max_index
# pivot_table[max_index]




id                                               ...            \
client_id   123  141  180  182  219  250  261  291  298  330   ... 1007 1017   
salesman_id                                                    ...             
36             7    2    6    2    5   18    4    1    6    2  ...    2    2   

                                                     
client_id   1064 1074 1079 1082 1097 1102 1115 1126  
salesman_id                                          
36             4    7    1    1   18   10    2    8  

[1 rows x 72 columns]

In [11]:
client_per_salesman.loc[client_per_salesman["client_id"] == 250]

,id,income,client_id,salesman_id,year,month,day
10,00034256,3542.50,250,36,2018,11,12
16,00034358,7010.67,250,36,2018,11,27
111,00034395,1642.85,250,36,2018,12,3
194,00034495,5483.82,250,36,2018,12,17
220,00034547,1126.44,250,36,2019,1,11
239,00034496,1024.65,250,36,2018,12,17
296,00034602,3138.31,250,36,2019,1,21
302,00034620,6287.82,250,36,2019,1,28
490,00034825,736.32,250,36,2019,4,5
564,00034978,1325.11,250,36,2019,5,8
